# MadMiner example

In this tutorial we'll demonstrate how to use MadMiner to generate train and test samples for the ML methods introduced in ["Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00013) and ["A Guide to Constraining Effective Field Theories With Machine Learning"](https://arxiv.org/abs/1805.00020), both by Johann Brehmer, Gilles Louppe, Juan Pavez, and Kyle Cranmer.

Before you execute this notebook, make sure you have running installations of MadGraph, Pythia, and Delphes. Note that at least for now, the MG-Pythia interface and Delphes require custom patches (available upon request). In addition, MadMiner and [DelphesMiner](https://github.com/johannbrehmer/delphesminer) have to be in your PYTHONPATH.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline

from madminer.goldmine import GoldMine
from madminer.refinery import combine_and_shuffle
from madminer.refinery import Refinery
from madminer.refinery import constant_benchmark_theta, multiple_benchmark_thetas
from madminer.refinery import constant_morphing_theta, multiple_morphing_thetas, random_morphing_thetas
from madminer.tools.plots import plot_2d_morphing_basis
from delphesprocessor.delphesprocessor import DelphesProcessor

Please enter here the path to your MG5 root directory. This notebook assumes that you installed Delphes and Pythia through MG5.

In [2]:
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2'

## 1. Define parameter space

After creating a `GoldMine` instance, the first important step is the definition of the parameter space. Each model parameter is characterized by a name as well as the LHA block and ID.

If morphing is used, one also has to specify the maximal power with which the parameter contributes to the squared matrix element. For instance, a parameter that contributes only to one vertex, will typically have `morphing_max_power=2`, while a parameter that contributes to two vertices usually has `morphing_max_power=4`. Exceptions arise for instance when the interference effects between the SM and dimension-six operators are modelled, but the square of the dimension-six amplitude (subleading in 1/Lambda) is not taken into account, in which case `morphing_max_power=1`. Finally, the `parameter_range` argument defines the range of parameter values that are used for the automatic optimization of the morphing basis.

In [ ]:
miner = GoldMine()

miner.add_parameter(
    lha_block='dim6',
    lha_id=2,
    parameter_name='CWL2',
    morphing_max_power=2,
    parameter_range=(-10.,10.)
)
miner.add_parameter(
    lha_block='dim6',
    lha_id=5,
    parameter_name='CPWL2',
    morphing_max_power=2,
    parameter_range=(-10.,10.)
)

## 2. Define benchmark points (evaluation points for |M|^2)

The next step is the definition of all the points at which the weights (squared matrix elements) should be evaluated by MadGraph. We call these points "benchmarks".

### 2a. Set benchmarks by hand

One can define benchmarks by hand:

In [ ]:
miner.add_benchmark(
    {'CWL2':0., 'CPWL2':0.},
    'sm'
)
miner.add_benchmark(
    {'CWL2':1., 'CPWL2':0.},
    'bsm'
)

### 2b. Benchmarks for morphing

If morphing is used, the function `set_benchmarks_from_morphing` has to be called. With the option `keep_existing_benchmarks=True`, MadMiner will keep all the benchmark points defined beforehand and run a simple optimization algorithm to fix the remaining ones for the basis (which may be none). Otherwise, MadMiner will optimize the full basis and forget about all previously defined benchmark points. The argument `n_trials` determines the number of random candidate bases that the optimization algorithm goes through.

In [ ]:
miner.set_benchmarks_from_morphing(
    keep_existing_benchmarks=True,
    n_trials=1000,
    max_overall_power=2
)

Let's have a look at the resulting morphing basis and the "morphing error", i.e. the sum of squared morphing weights as a function of the parameter space:

In [ ]:
fig = plot_2d_morphing_basis(
    miner.morpher,
    xlabel=r'$c_{W} / \Lambda^2$ [TeV$^{-2}$]',
    ylabel=r'$c_{\tilde{W}} / \Lambda^2$ [TeV$^{-2}$]',
    xrange=(-10.,10),
    yrange=(-10.,10.)
)

## 3. Save settings and run MadGraph

The parameter space, benchmark points, and morphing setup are saved in a HDF5 file:

In [ ]:
miner.save('data/madminer_example.h5')

In a next step, MadMiner starts MadGraph and Pythia to generate events and calculate the weights. You have to provide paths to the process card, run card, param card (the entries corresponding to the parameters of interest will be automatically adapted), and an empty reweight card.

The `sample_benchmark` option can be used to specify which benchmark should be used for sampling. If it is not used, MadMiner will automatically use the benchmark that was added first. Finally, if MadGraph is supposed to run in a different Python environment or requires other setup steps, you can use the `initial_command` argument.

In [ ]:
miner.run(
    mg_directory=mg_dir,
    proc_card_file='cards/proc_card.dat',
    param_card_template_file='cards/param_card_template.dat',
    reweight_card_template_file='cards/reweight_card_template.dat',
    run_card_file='cards/run_card.dat',
    pythia8_card_file='cards/pythia8_card.dat',
    sample_benchmark='sm',
    initial_command='source activate python2'
)

## 4. Run detector simulation and extract observables

The detector simulation and calculation of observables is not part of MadMiner. The reason is that different users might have very different requirements here: while a phenomenologist might be content with the fast detector simulation from Delphes, an experimental analysis might require the full simulation through Geant4.

We provide the DelphesMiner package, which wraps around Delphes and allows for the fast extraction of observables into the HDF5 file.

Any user is free to replace the DelphesMiner step with a tool of their choice. 

In [ ]:
dm = DelphesProcessor()

After creating the DelphesProcessor object, one can add a number of HepMC event samples...

In [ ]:
dm.add_hepmc_sample('MG_process/Events/run_01/tag_1_pythia8_events.hepmc.gz')

... and have it run Delphes:

In [ ]:
dm.run_delphes(delphes_directory=mg_dir + '/Delphes',
               delphes_card='cards/delphes_card.dat',
               initial_command='source activate python2')

The next step is the definition of observables through a name and a python expression. For the latter, you can use the objects `j[i]`, `e[i]`, `mu[i]`, `a[i]`, `met`, where the indices `i` refer to a ordering by the transverse momentum. All of these objects are scikit-hep [LorentzVectors](http://scikit-hep.org/api/math.html#vector-classes), see the link for a documentation of their properties.

There is an optional keyword `required`. If `required=True`, we will only keep events where the observable can be parsed, i.e. all involved particles have been detected. If `required=False`, un-parseable observables will be filled with `np.nan`.

In [ ]:
dm.add_observable('pt_e1', 'e[0].pt', required=True)
dm.add_observable('pt_mu1', 'mu[0].pt', required=True)
dm.add_observable('delta_eta_ll', 'abs(e[0].eta - mu[0].eta)', required=True)
dm.add_observable('delta_phi_ll', 'abs(e[0].phi() - mu[0].phi())', required=True)

The function `analyse_delphes_samples` extracts all these observables from the Delphes ROOT file(s) generated before:

In [ ]:
dm.analyse_delphes_samples()

The values of the observables and the weights are then saved in the HDF5 file. It is possible to overwrite the same file, or to leave the original file intact and save all the data into a new file as follows:

In [ ]:
dm.save('data/madminer_example_with_data.h5', 'data/madminer_example.h5')

It's easy to check some distributions at this stage

In [ ]:
fig = plt.figure(figsize=(5,5))

for weights in dm.weights:
    plt.hist(dm.observations['pt_e1'], range=(0.,400.), bins=20, histtype='step', weights=weights)

plt.show()

## 5. Combine and shuffle different event samples

To reduce disk usage, you can generate several small event samples with the steps given above, and combine them now. Note that (for now) it is essential that all of them are generated with the same setup, including the same benchmark points / morphing basis!

In [ ]:
combine_and_shuffle(
    ['data/madminer_example_with_data.h5'],
    'data/madminer_example_shuffled.h5'
)

## 6. Make (unweighted) training and test samples

The last important MadMiner class is the `Smithy`. From all the data we have in the HDF5 file now, it extracts unweighted samples including the augmented data ("gold") that is needed as training and evaluation data for the Machine Learning algorithms.

In [3]:
refinery = Refinery('data/madminer_example_shuffled.h5', debug=True)

22:52  
22:52  ------------------------------------------------------------
22:52  |                                                          |
22:52  |  MadMiner                                                |
22:52  |                                                          |
22:52  |  Version from July 19, 2018                              |
22:52  |                                                          |
22:52  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
22:52  |                                                          |
22:52  ------------------------------------------------------------
22:52  
22:52  
    
                                                      @ @ @ @   @ @ @ @                                
                                                     @. . . . @ . . . . @                              
                                                     @. . . . . . . . . @                              
                   @@@@@@@@@@@@@@@             

The `Refinery` class defines four different high-level functions to generate train or test samples:
- `extract_samples_train_plain()`, which only saves observations x, for instance for histograms or ABC;
- `extract_samples_train_local()` for methods like SALLY and SALLINO;
- `extract_samples_train_ratio()` for techniques like CARL, ROLR, CASCAL, and RASCAL; and
- `extract_samples_test()` for the evaluation of any method.

For the arguments `theta`, `theta0`, or `theta1`, you can use the helper functions `constant_benchmark_theta()`, `multiple_benchmark_thetas()`, `constant_morphing_theta()`, `multiple_morphing_thetas()`, and `random_morphing_thetas()`, all defined in the `smithy` module.

In [4]:
x, theta, t_xz = refinery.extract_samples_train_local(
    theta=constant_morphing_theta(np.array([1.e-5,0.])),
    n_samples=100000,
    folder='./data/samples',
    filename='train_sally'
)

22:52  Extracting training sample for local score regression. Sampling and score evaluation according to ('theta', array([1.e-05, 0.e+00]))
22:52  Starting sample extraction
22:52  New theta configuration
22:52    Sampling theta: morphing, [1.e-05 0.e+00]
22:52    Auxiliary theta: None, None
22:52    # samples: 100000


In [ ]:
x, theta0, theta1, y, r_xz, t_xz = refinery.extract_samples_train_ratio(
    theta0=random_morphing_thetas(None, [('gaussian', 0., 0.5), ('flat', -0.8, 0.8)]),
    theta1=constant_benchmark_theta('sm'),
    n_samples=100000,
    folder='./data/samples',
    filename='train_rascal'
)

22:52  Extracting training sample for ratio-based methods. Numerator hypothesis: ('random', (None, [('gaussian', 0.0, 0.5), ('flat', -0.8, 0.8)])), denominator hypothesis: ('benchmark', 'sm')
22:52  Starting sample extraction
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.05963082 -0.07723056]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.15310431 -0.39404116]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.7979438  -0.34352131]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.7865885  -0.55318764]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.79654597  0.27568085]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:5

22:52  New theta configuration
22:52    Sampling theta: morphing, [0.73391565 0.54886726]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.30711211 -0.01657287]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.15712562 -0.784902  ]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.382593    0.72977557]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [0.04865845 0.05827624]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [0.06104365 0.26090006]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [0.18330461 0.58

22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.47351902 -0.12069316]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.48308195  0.79686283]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.18665107 -0.66622267]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.37434078 -0.76324642]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.03355229 -0.65662621]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [0.19594974 0.69742347]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta

22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.06693435 -0.53399456]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.58399042  0.11544255]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [0.40683933 0.79721775]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.1977764  -0.59038934]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.2310953  -0.32196766]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.48706443 -0.68709472]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.20680797

22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.21358427 -0.18276065]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [0.7733566  0.03257123]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [0.58450549 0.75710708]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.16964379 -0.4877268 ]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [0.19069656 0.38832319]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.60289339 -0.42406357]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta con

...

22:52    Sampling theta: morphing, [ 0.65362884 -0.38973405]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [0.34845983 0.06727002]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [ 0.16318497 -0.44704851]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [0.40160606 0.28278097]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.42252428  0.55657489]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.97554403  0.03616668]
22:52    Auxiliary theta: benchmark, sm
22:52    # samples: 1
22:52  New theta configuration
22:52    Sampling theta: morphing, [-0.18759293 -0.18440471]
22:52    Auxiliary 

22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [0.13104524 0.42653409]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-0.15209095 -0.68397198]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-0.05188342 -0.61005745]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-0.43057577  0.22398733]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [ 0.0640943  -0.52888071]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [1.03428686 0.5816271 ]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta c

22:53    Sampling theta: morphing, [-0.12054761 -0.39919048]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-0.78541523  0.25919756]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-0.89083095  0.60357898]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-0.10970091  0.32148037]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [ 0.53010854 -0.72515637]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [0.14582189 0.22937411]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-0.34251907 -0.49515063]
22:53    Auxiliar

22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [0.46084466 0.40050625]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [0.3839782  0.28182502]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-0.28287292  0.28600489]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-0.42684256  0.09419492]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [ 0.00952707 -0.50128468]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [ 0.51990736 -0.04807512]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: mo

22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [0.38037569 0.06525409]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-0.28351803  0.27409647]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [-1.26193798 -0.10076505]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [ 0.66833497 -0.58928549]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [0.1585985  0.69128237]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta configuration
22:53    Sampling theta: morphing, [1.14343427 0.01314661]
22:53    Auxiliary theta: benchmark, sm
22:53    # samples: 1
22:53  New theta con

In [ ]:
x, theta = refinery.extract_samples_test(
    theta=constant_benchmark_theta('sm'),
    n_samples=100000,
    folder='./data/samples',
    filename='test'
)

Let's have a look at some distributions and correlations in this test sample:

In [ ]:
import corner

labels = [r'$p_{T,e}$ [GeV]', r'$p_{T,\mu}$ [GeV]', r'$\Delta \eta_{\ell\ell}$', r'$\Delta \phi_{\ell\ell}$']
ranges = [(0., 500.), (0., 500.), (0.,3.), (0.,6.2)]

_ = corner.corner(x, color='C0', labels=labels, range=ranges)